In [66]:
import pandas as pd
import numpy as np

In [67]:
df_extract = pd.read_excel('results/result_extraction.xlsx')
df_gemini = pd.read_csv("results/responses_subset_small_all_results_updated.csv")
df_lens_t5 = pd.read_csv('results_lens/SA_results_lens_t5_subset_5000.csv')
df_vqa_base = pd.read_csv('results_VQA/SA_results_blip-vqa-base-large_subset_5000.csv')
df_vqa_capfilt = pd.read_csv('results_VQA/SA_results_blip-vqa-capfilt-large_subset_5000.csv')
df_text = pd.read_csv('results_text/results_text_roBERTa.csv')

In [68]:
dfs = [df_extract, df_gemini, df_lens_t5, df_vqa_base, df_vqa_capfilt, df_text]
df_names = ['df_extract', 'df_gemini', 'df_lens_t5', 'df_vqa_base', 'df_vqa_capfilt', 'df_text']

for i in range(len(dfs)):
    print(f"{df_names[i]}: {list(dfs[i].columns)}")
    print()

df_extract: ['image_name', 'object_detection_results', 'aisak_description', 'moondream_description', 'gemini_description', 'facial_emotion', 'confidence']

df_gemini: ['image', 'sentiment', 'confidence', 'explanation']

df_lens_t5: ['Unnamed: 0', 'image', 'sentiment']

df_vqa_base: ['Unnamed: 0', 'image', 'sentiment']

df_vqa_capfilt: ['Unnamed: 0', 'image', 'sentiment']

df_text: ['created_at', 'img_name', 'language', 'like_count', 'quote_count', 'referenced_tweets', 'retweet_count', 'text', 'tweet_id', 'Negative', 'Neutral', 'Positive', 'text_sent']



In [69]:
# rename column sentiment --> model

df_extract = df_extract.rename(columns={'image_name': 'image'})
df_gemini = df_gemini.rename(columns={'sentiment': 'gemini_img_sentiment'})
df_lens_t5 = df_lens_t5.rename(columns={'sentiment': 'lens_t5_img_sentiment'})
df_vqa_base = df_vqa_base.rename(columns={'sentiment': 'vqa_base_img_sentiment'})
df_vqa_capfilt = df_vqa_capfilt.rename(columns={'sentiment': 'vqa_capfilt_img_sentiment'})
df_text = df_text.rename(columns={'text_sent': 'roberta_text_sentiment'})
df_text = df_text.rename(columns={'img_name': 'image'})

In [70]:
print(df_gemini.duplicated().sum())
df_gemini = df_gemini.drop_duplicates()
print(df_gemini.duplicated().sum())


3000
0


In [71]:
merged_df = df_extract.copy()

merged1 = pd.merge(merged_df, df_gemini[['image', 'gemini_img_sentiment']], on='image', how='left')
merged2 = pd.merge(merged1, df_lens_t5[['image', 'lens_t5_img_sentiment']], on='image', how='left')
merged3 = pd.merge(merged2, df_vqa_base[['image', 'vqa_base_img_sentiment']], on='image', how='left')
merged4 = pd.merge(merged3, df_vqa_capfilt[['image', 'vqa_capfilt_img_sentiment']], on='image', how='left')
merged5 = pd.merge(merged4, df_text[['image', 'like_count', 'quote_count', 'retweet_count', 'roberta_text_sentiment']], on='image', how='left')

merged5.head()

,image,object_detection_results,aisak_description,moondream_description,gemini_description,facial_emotion,confidence,gemini_img_sentiment,lens_t5_img_sentiment,vqa_base_img_sentiment,vqa_capfilt_img_sentiment,like_count,quote_count,retweet_count,roberta_text_sentiment
0,id_1153283149360762880_2019-07-22.jpg,NaN,a screenshot of a cell phone with a message ab...,The image shows a screenshot of a webpage from...,The image is a screenshot of a web page. The ...,---,0.873518,negative,negative,positive,positive,82582,3918,50280,Negative
1,id_1163744643600637952_2019-08-20.jpg,NaN,trees are in the background,The image depicts a serene winter landscape wi...,There are 15 trees on the right side of the i...,---,0.906822,neutral,positive,positive,positive,69820,2456,51781,Negative
2,id_1122574040936452097_2019-04-28.jpg,"bird,",a close up of a hole in a wall with a black an...,"The image shows a hole in a white wall, with a...",This is an image of a hole in a wall. The hol...,---,0.903128,negative,negative,neutral,neutral,69235,87,11051,Neutral
3,id_1188805167958974465_2019-10-28.jpg,"person,",araffe in a red suit falling down a snowy ramp,"In the image, a person dressed in a vibrant re...","The image shows a man in a purple suit, green...",disgust,0.881499,negative,negative,negative,negative,65465,70,6124,Negative
4,id_1108042949449969666_2019-03-19.jpg,NaN,spongebob squarepants - the tree in the middle...,The image depicts a vibrant underwater scene f...,The image is a cartoon. It shows a tree insid...,---,0.916996,neutral,negative,positive,positive,62852,976,9145,Negative


In [72]:
merged5.shape

(5000, 15)

In [73]:
merged5.isna().sum()

image                           0
object_detection_results     1405
aisak_description               0
moondream_description           0
gemini_description            636
facial_emotion                  0
confidence                      0
gemini_img_sentiment            0
lens_t5_img_sentiment           0
vqa_base_img_sentiment          0
vqa_capfilt_img_sentiment       0
like_count                      0
quote_count                     0
retweet_count                   0
roberta_text_sentiment         30
dtype: int64

In [74]:
# export dataframes
merged5.to_excel('results/merged_results.xlsx', index=False)